## KerasによるCIFAR10のGeneratorサンプル
KerasによるCIFAR10のCNN実装のデータ読み込みを、Generator経由にしたsample
fit_generator実装

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.datasets import cifar10

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

from keras import backend as K



Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 12

In [3]:
img_rows, img_cols = 32, 32

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
input_shape = (img_rows, img_cols, 3)


In [5]:
x_train  = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation="softmax"))

In [8]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adadelta(),
             metrics=["accuracy"])

In [9]:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))

Epoch 1/12
390/390 [==============================] - 279s - loss: 1.8612 - acc: 0.3250 - val_loss: 1.4528 - val_acc: 0.4984
Epoch 2/12
390/390 [==============================] - 293s - loss: 1.5092 - acc: 0.4565 - val_loss: 1.2439 - val_acc: 0.5605
Epoch 3/12
390/390 [==============================] - 254s - loss: 1.3800 - acc: 0.5077 - val_loss: 1.1289 - val_acc: 0.6067
Epoch 4/12
390/390 [==============================] - 254s - loss: 1.2908 - acc: 0.5448 - val_loss: 1.0824 - val_acc: 0.6232
Epoch 5/12
390/390 [==============================] - 300s - loss: 1.2343 - acc: 0.5641 - val_loss: 1.0363 - val_acc: 0.6421
Epoch 6/12
390/390 [==============================] - 253s - loss: 1.1808 - acc: 0.5855 - val_loss: 1.0119 - val_acc: 0.6483
Epoch 7/12
390/390 [==============================] - 251s - loss: 1.1442 - acc: 0.6018 - val_loss: 0.9765 - val_acc: 0.6642
Epoch 8/12
390/390 [==============================] - 8585s - loss: 1.1175 - acc: 0.6089 - val_loss: 0.9592 - val_acc: 0.6642

In [10]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.854277688408
Test accuracy: 0.7063
